In [1]:
# 加载data、label
import scipy.io as io
data_mat_path = 'mats/'
label_mat_path = 'labels/'

data_filename = 'data_old.mat'
label_filename = 'label_old.mat'
data = io.loadmat(data_mat_path+data_filename)["name"]
label = io.loadmat(label_mat_path+label_filename)["name"]

In [2]:
label = io.loadmat(label_mat_path+label_filename)["name"]

In [3]:
for i in range(64467):  # 去除因对齐添加的空格
    for j in range(16):
        label[i][j] = label[i][j].strip()

In [4]:
label[88]

array(['Acura', 'ILX', '2017', '27', '17', '200', '24', '4', '70', '55',
       '181', '25', 'FWD', '5', '4', '4dr'], dtype='<U21')

In [5]:
# 取shape一列
row=len(label)
y=['' for i in range(row)]
for i in range(row):
    y[i]=label[i][15]


In [6]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
# y = encoder.fit_transform(y)

# 训练

In [ ]:
from sklearn.preprocessing import LabelEncoder
y_one_hot = y
encoder2 = LabelEncoder()
y2 = encoder2.fit_transform(y)

In [7]:
# 划分训练集测试集
from sklearn.model_selection import train_test_split
X_bs_train, X_bs_test, y_bs_train, y_bs_test = train_test_split(
    data, y, test_size=0.2)

In [8]:
# 归一化
X_bs_train=X_bs_train/255
X_bs_test=X_bs_test/255

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [10]:
row,height, width, channel=data.shape
input_shape = (height, width, channel)

In [11]:
input_shape

(60, 80, 3)

In [12]:
# 减小复杂度解决过拟合
# 增加dropout
# 构造网络

bodyStyleModel = Sequential()
# 第一层
bodyStyleModel.add(Conv2D(filters=32,
                          input_shape=input_shape,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
# 中间各层
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(5, 5),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )

bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.3))
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(5, 5),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(3, 3),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.3))
bodyStyleModel.add(Flatten())
bodyStyleModel.add(Dense(512, activation='relu'))
bodyStyleModel.add(Dropout(0.4))
bodyStyleModel.add(Dense(len(y[0]), activation='softmax'))



In [13]:
from tensorflow.keras.optimizers import SGD

learning_rate = 0.01
decay = 1e-6
momentum = 0.8
nesterov = True
sgd_optimizer = SGD(lr=learning_rate, decay=decay,
                    momentum=momentum, nesterov=nesterov)

In [14]:
bodyStyleModel.compile(optimizer=sgd_optimizer,
                   loss="categorical_crossentropy",
                   metrics=['accuracy']
                   )
batch_size = 50  # 每批训练数据量的大小
epochs = 100
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 51573 samples
Epoch 1/100
51573/51573 [==============================] - 22s 418us/sample - loss: 1.6237 - accuracy: 0.3434
Epoch 2/100
51573/51573 [==============================] - 16s 310us/sample - loss: 1.5095 - accuracy: 0.4157
Epoch 3/100
51573/51573 [==============================] - 16s 311us/sample - loss: 1.3787 - accuracy: 0.4731
Epoch 4/100
51573/51573 [==============================] - 17s 334us/sample - loss: 1.2907 - accuracy: 0.5057
Epoch 5/100
51573/51573 [==============================] - 17s 321us/sample - loss: 1.2327 - accuracy: 0.5290
Epoch 6/100
51573/51573 [==============================] - 16s 317us/sample - loss: 1.1919 - accuracy: 0.5456
Epoch 7/100
51573/51573 [==============================] - 17s 336us/sample - loss: 1.1577 - accuracy: 0.5585
Epoch 8/100
51573/51573 [==============================] - 18s 342us/sample - loss: 1.1250 - accuracy: 0.5695
Epoch 9/100
51573/51573 [==============================] - 18s 342us/sample - loss: 1.0994 - accu

51573/51573 [==============================] - 17s 336us/sample - loss: 0.4960 - accuracy: 0.8179
Epoch 75/100
51573/51573 [==============================] - 17s 322us/sample - loss: 0.4944 - accuracy: 0.8181
Epoch 76/100
51573/51573 [==============================] - 17s 329us/sample - loss: 0.4916 - accuracy: 0.8203
Epoch 77/100
51573/51573 [==============================] - 17s 327us/sample - loss: 0.4928 - accuracy: 0.8178
Epoch 78/100
51573/51573 [==============================] - 17s 322us/sample - loss: 0.4810 - accuracy: 0.8221
Epoch 79/100
51573/51573 [==============================] - 17s 324us/sample - loss: 0.4809 - accuracy: 0.8246
Epoch 80/100
51573/51573 [==============================] - 17s 324us/sample - loss: 0.4828 - accuracy: 0.8253
Epoch 81/100
51573/51573 [==============================] - 17s 321us/sample - loss: 0.4790 - accuracy: 0.8268
Epoch 82/100
51573/51573 [==============================] - 17s 324us/sample - loss: 0.4713 - accuracy: 0.8279
Epoch 83/100
5

In [ ]:
MODEL_PATH = 'model/bodystyle_model_85.h5'
bodyStyleModel.save(MODEL_PATH)

# 字典

In [15]:
y_one_hot = y

In [19]:
label = io.loadmat(label_mat_path+label_filename)["name"]
for i in range(64467):  # 去除因对齐添加的空格
    for j in range(16):
        label[i][j] = label[i][j].strip()
# 取shape一列
row=len(label)
y=['' for i in range(row)]
for i in range(row):
    y[i]=label[i][15]

In [27]:
# for items in y:
#     if items in ('2dr','4dr','SUV','Convertible','Pickup','Station Wagon','Van','nan'):
#         pass
#     else:
#         print(i,items)

In [22]:
len(y_one_hot[0])

9

In [ ]:
# 保存到文件 避免下次重复计算
import scipy.io as io
dict_mat_path = 'dicts/bodystyle'
io.savemat(dict_mat_path, {'name': dict_bs})
